In [50]:
import os
import sys

# Set the root directory of your project
project_root = '/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_project'
os.chdir(project_root)
# Add the root directory to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from config import (CONFIG_SQL_DB,DB_NAME,
                    SQL_EVAL_QAS_TABLE_SCHEMA,
                    SQL_EVAL_QAS_TABLE, 
                    EMBEDDING_MODEL,EMBEDDING_MODEL_API,
                    EMBEDDING_MODEL_EMB_TASK,
                    TEST_RESULTS_TABLE,SQL_EVAL_CHUNKS_TABLE,
                    SQL_PROMPTS_TABLE)
from utils.MySQLDB_manager import MySQLDB
from testing.modules.evaluating_modules import RAGEvaluator
from utils.embedding_handler import EmbeddingHandler
import json

TEST_NAME = "testJinaDE500"

sql_con = MySQLDB(CONFIG_SQL_DB,DB_NAME)

ragEval = RAGEvaluator(sql_con=sql_con,
                       test_name=TEST_NAME,
                       test_table_name=TEST_RESULTS_TABLE,
                       qas_table_name=SQL_EVAL_QAS_TABLE,
                       chunks_eval_table_name=SQL_EVAL_CHUNKS_TABLE,
                       prompts_table_name=SQL_PROMPTS_TABLE)

df_results = ragEval.data_df

print(df_results.head(5))
print(df_results.columns)
print(df_results.shape)

print(ragEval.generate_report())


KeyError: 'device'

In [49]:
def get_adj_ids(id, last_id, dif=1):
    """
    Get all adjacent IDs within a given difference from the original ID.
    
    Args:
        id (str): The current ID in the format "base.cid".
        last_id (str): The last ID in the format "base.lid".
        dif (int): The difference to consider for adjacent IDs.

    Returns:
        list: A list of adjacent IDs within the specified range.
    """
    adjs_ids = []

    # Split and parse the base and numeric parts
    base, cid = id.split('.')
    _, lid = last_id.split('.')

    # Convert to integers
    cid = int(cid)
    lid = int(lid)

    # Generate all IDs within the range [cid-dif, cid+dif]
    for offset in range(-dif, dif + 1):
        adj_cid = cid + offset
        # Ensure IDs are within bounds and not the original ID
        if 0 <= adj_cid <= lid and adj_cid != cid:
            adjs_ids.append(f"{base}.{adj_cid}")

    return adjs_ids


def evaluate_context(df):

    used_count = 0
    retrieved_count = 0

    used_exp_count = 0
    retrieved_exp_count = 0

    for index,row in df.iterrows():
        metadata = json.loads(row['metadata'])
        end_chunk = metadata['last_id']
        context_ids = json.loads(row['context_ids'])
        total_context_ids = json.loads(row['context_ids_total'])
        original_chunk_id = row['id']
        adj_context = get_adj_ids(original_chunk_id,end_chunk,dif=1)
        adj_context.append(original_chunk_id)
        used_context = original_chunk_id in context_ids
        retrieved_context = original_chunk_id in total_context_ids
        
        used_context_ext = any(item in adj_context for item in context_ids) 
        retrieved_context_ext = any(item in adj_context for item in total_context_ids)

        used_count+=used_context
        retrieved_count+=retrieved_context
        used_exp_count+=used_context_ext
        retrieved_exp_count+=retrieved_context_ext

        # print(f"Index: {index} Used: {used_context} Retrieved: {retrieved_context}")

    print(f"Used: {used_count/len(df)} Used Ext: {used_exp_count/len(df)}\nRetrieved: {retrieved_count/len(df)} Retrieved Ext: {retrieved_exp_count/len(df)}")


    return 0


res = evaluate_context(df_results)
print(res)

Used: 0.027848101265822784 Used Ext: 0.0379746835443038
Retrieved: 0.0810126582278481 Retrieved Ext: 0.15443037974683543
0


In [34]:
adjies = get_adj_ids("0.5","0.100",4)
adjies.append('0.5')
print(adjies)

['0.1', '0.2', '0.3', '0.4', '0.6', '0.7', '0.8', '0.9', '0.5']


In [ ]:
testJinaV3-250
Used: 0.027848101265822784 Used Ext: 0.0379746835443038
Retrieved: 0.0810126582278481 Retrieved Ext: 0.15443037974683543


testJinaV3-500
Used: 0.41265822784810124 Used Ext: 0.5341772151898734
Retrieved: 0.579746835443038 Retrieved Ext: 0.6481012658227848